## Demand Estimation Exercise

In this assignment, you will learn how to simulate market data and estimate simple random coefficient demand models. The assignment should serve as an introduction to the structural estimation in heterogenous product demand models.

Let's first define the model.


In [1]:
import simulate
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
market = simulate.Market_Data(10, 1000, 3, 10)
print(market)

Market with 10 firms and 1000 consumers over 10 time periods. 
 Firms sell differentiated product which have 3 product characteristics


In [6]:
print(market.produc_chars.shape)
print(market.produc_chars)

print(market.prices)
print(market.prices.shape)





(10, 3)
[[5.27222258 2.05193222 1.38320935]
 [4.94457399 3.73750001 4.93127431]
 [5.60023521 3.40486383 3.29776835]
 [3.99489578 3.9965939  3.52186726]
 [2.53439265 3.7067649  5.62463472]
 [5.85275401 2.97897305 4.99372636]
 [4.17544074 2.14984583 1.25603546]
 [1.14231903 1.61423876 2.10106259]
 [5.14511377 2.42745914 4.90532041]
 [3.52332906 1.69224462 4.89018276]]
[[ 0.169921  ]
 [ 0.86102689]
 [ 0.00513994]
 [ 0.03529768]
 [ 2.86457211]
 [ 1.22727149]
 [ 0.10537246]
 [-0.16729861]
 [ 0.9387192 ]
 [-0.13748932]
 [-0.67818232]
 [ 0.76239127]
 [ 1.19493066]
 [ 1.09266934]
 [ 1.01582885]
 [-0.70639973]
 [-0.25999155]
 [ 0.82469389]
 [-0.29318707]
 [ 0.1315218 ]
 [-0.05534718]
 [-0.76808008]
 [-0.83299498]
 [ 0.5135007 ]
 [ 0.74097149]
 [ 0.32817733]
 [ 2.59366079]
 [ 0.65289221]
 [-0.0964893 ]
 [ 1.94203607]
 [-0.1235615 ]
 [ 0.02339787]
 [ 0.99477643]
 [ 0.16701093]
 [ 1.6163454 ]
 [-0.61989536]
 [-0.30696023]
 [ 1.49153619]
 [-0.81029856]
 [ 0.03034852]
 [-0.36501127]
 [ 0.33590149]
 